In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# third party
# run email server
from helpers import EmailServer
from helpers import SMTPTestServer

email_server = EmailServer()
smtp_server = SMTPTestServer(email_server)
smtp_server.start()

In [ ]:
# syft absolute
import syft as sy

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
# third party
from helpers import load_users
from job_helpers import load_jobs
from job_helpers import save_jobs

In [ ]:
users = load_users(high_client)

In [ ]:
jobs = load_jobs(users, high_client)

In [ ]:
# submitted_jobs = [job for job in jobs if job.is_submitted]
reviewed_jobs = [job for job in jobs if job.admin_reviewed]

In [ ]:
# TODO: test jobs that were never approved
# they seem to give weird errors like
# "You uploaded an ActionObject that is not yet in the blob storage"

In [ ]:
for job in reviewed_jobs:
    print(f"> Checking job: {job.job_type} {job.func_name} for user {job.user_email}")
    api_method = job.code_method

    if job.should_succeed:
        print("Expecting job to succeed")
        j = api_method(blocking=False)
        result = j.wait().get()
        print(
            "> Got result of type",
            type(result),
            "expecting limit",
            job.settings["limit"],
        )
        if hasattr(result, "__len__"):
            print("> Result length", len(result))
        assert len(result) == job.settings["limit"]
        job.result_as_expected = True
        save_jobs(jobs)
    else:
        print("Expecting job to fail")
        # with sy.raises(
        #     sy.SyftException(public_message="*UserCodeStatus.DENIED*"), show=True
        # ):
        # with sy.raises(
        #     sy.SyftException, show=True
        # ):
        try:
            j = api_method(blocking=False)
            print("failed job didnt raise", type(j))
            job.result_as_expected = False
        except Exception as e:
            job.result_as_expected = True
            print("failed job raised", type(e))

        save_jobs(jobs)

In [ ]:
expected_jobs = [job for job in jobs if job.result_as_expected]

In [ ]:
len(expected_jobs)

In [ ]:
# TODO fix
# assert len(reviewed_jobs) == len(expected_jobs)

In [ ]:
smtp_server.stop()

In [ ]:
server.land()